In [2]:
import random
import numpy as np
import pandas as pd
import datetime as dt
#import matplotlib.pyplot as plt
#import seaborn as sns
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import dbscan
from sklearn.cluster import k_means
from sklearn.cluster import KMeans
from tqdm import tqdm_notebook
from collections import Counter

#plt.rcParams['font.family'] = 'NanumGothic'
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:.3f}'.format

In [10]:
train = pd.read_json('train.json', typ = 'frame', encoding='utf-8')
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


하나의 플레이리스트에 태그가 여러 개면 그 태그를 각각 한 row로 펼쳐야 함.

In [61]:
# 플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_tag_map = train[['id', 'tags']]

# unnest songs
plylst_tag_map_unnest = np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))), 
        np.concatenate(plylst_tag_map.tags.values)
    )
)

# unnested 데이터프레임 생성 : plylst_tag_map
plylst_tag_map = pd.DataFrame(data = plylst_tag_map_unnest[0], columns = plylst_tag_map.columns)
#plylst_tag_map['id'] = plylst_tag_map['id'].astype(str)
plylst_tag_map['id'] = plylst_tag_map['id'].astype('int')
plylst_tag_map['tags'] = plylst_tag_map['tags'].astype(str)

# unnest 객체 제거
del plylst_tag_map_unnest

In [62]:
plylst_tag_map

,id,tags
0,61281,락
1,10532,추억
2,10532,회상
3,76951,까페
4,76951,잔잔한
...,...,...
476326,131982,퇴근길
476327,100389,노래추천
476328,100389,팝송추천
476329,100389,팝송


### 잠시 전처리를 해볼까 생각함

In [43]:
plylst_tag_map1 = plylst_tag_map.copy()

In [44]:
# 이렇게 전처리를 해버리면 나중에 예측할 때는 바꿔준 애도 같이 넣어버리면 되나..
plylst_tag_map1['tags'] = plylst_tag_map1['tags'].apply(lambda x: '카페' if x=='까페' else x)

In [47]:
plylst_tag_map1

,id,tags
0,61281,락
1,10532,추억
2,10532,회상
3,76951,카페
4,76951,잔잔한
...,...,...
476326,131982,퇴근길
476327,100389,노래추천
476328,100389,팝송추천
476329,100389,팝송


In [48]:
plylst_tag_map1['tags'] = plylst_tag_map1['tags'].apply(lambda x: '캐롤' if x=='캐럴' else x)

In [50]:
plylst_tag_map1[plylst_tag_map1['tags']=='캐롤']

,id,tags
7,147456,캐롤
181,43665,캐롤
785,22729,캐롤
1067,3526,캐롤
1595,47623,캐롤
...,...,...
473793,75699,캐롤
474663,91612,캐롤
474906,97036,캐롤
475139,112367,캐롤


### 연관분석하는 이유 중 하나가 유사한 단어들(까페, 카페) 한 곳에 묶기 위한 것도 있으니 우선 전처리 안 한 데이터로 고고

In [63]:
plylst_tag_map

,id,tags
0,61281,락
1,10532,추억
2,10532,회상
3,76951,까페
4,76951,잔잔한
...,...,...
476326,131982,퇴근길
476327,100389,노래추천
476328,100389,팝송추천
476329,100389,팝송


In [64]:
trans_id_idx = plylst_tag_map.set_index('id')['tags']
trans_id_idx 

id
61281        락
10532       추억
10532       회상
76951       까페
76951      잔잔한
          ... 
131982     퇴근길
100389    노래추천
100389    팝송추천
100389      팝송
100389    팝송모음
Name: tags, Length: 476331, dtype: object

In [65]:
trans_id_idx.index.unique()

Int64Index([ 61281,  10532,  76951, 147456,  27616,  69252,  45339,  36557,
             70741,  10288,
            ...
             53627,   2531, 120713, 145872, 135687, 120325, 106976,  11343,
            131982, 100389],
           dtype='int64', name='id', length=115071)

In [66]:
item_tuple_to_cnt = {}
for i in tqdm_notebook(trans_id_idx.index.unique()): # 고유한 플레이리스트를 하나씩 뽑아냄. 그럼 해당 플스트에 속한 곡들이 나옴
    items_ = trans_id_idx.loc[i] # 그 플레이리스트에 속한 태그들을 저장
    if type(items_) is not str:
        visit = set()
        for j in range(len(items_)): # 그 플스트의 태그들 중 하나를 대상으로
            tmp = list(items_) # (태그들을 리스트로 만들고)
            item = tmp.pop(j) # (그 리스트에서 하나씩 튀어나오게 함)
            if item in visit: # 그 태그가 visit 집합 안에 있으면
                continue # 다시 첨으로
            visit.add(item)  # visit 집합 안에 포함되어 있지 않으면 추가함
            for k in set(tmp): # 그 플스트의 고유한 태그 내역에서 하나씩 뽑아냄
                item_tuple_to_cnt[(item,k)] = item_tuple_to_cnt.get((item,k), 0) + 1 # 둘이 같이 나온 수를 세는 것
                # 딕셔너리에 그 플스트의 태그들 중 하나랑, 그 플스트에 있는 태그 중 하나를 뽑아내서 key로 만들고,
                # 그 key에 대한 값을 return하고 1을 더함. 키에 대한 값이 없으면 0을 리턴하고 1을 더함
                # 그런 다음 플스트에서 똑같은 태그가 같이 또 나오면 계속 1이 더해지면서 동시 출현 횟수를 구하는 것.

In [67]:
item_tuple_to_cnt # 두 태그가 같이 나온 횟수

{('추억', '회상'): 4334,
 ('회상', '추억'): 4334,
 ('까페', '잔잔한'): 325,
 ('잔잔한', '까페'): 325,
 ('연말', '겨울노래'): 33,
 ('연말', '크리스마스송'): 1,
 ('연말', '눈오는날'): 11,
 ('연말', '크리스마스'): 186,
 ('연말', '분위기'): 36,
 ('연말', '캐럴'): 18,
 ('연말', '따듯한'): 12,
 ('연말', '크리스마스캐럴'): 1,
 ('연말', '겨울왕국'): 10,
 ('눈오는날', '겨울노래'): 11,
 ('눈오는날', '크리스마스송'): 1,
 ('눈오는날', '크리스마스'): 36,
 ('눈오는날', '분위기'): 6,
 ('눈오는날', '캐럴'): 2,
 ('눈오는날', '연말'): 11,
 ('눈오는날', '따듯한'): 5,
 ('눈오는날', '크리스마스캐럴'): 1,
 ('눈오는날', '겨울왕국'): 1,
 ('캐럴', '겨울노래'): 12,
 ('캐럴', '크리스마스송'): 1,
 ('캐럴', '눈오는날'): 2,
 ('캐럴', '크리스마스'): 58,
 ('캐럴', '분위기'): 6,
 ('캐럴', '따듯한'): 3,
 ('캐럴', '연말'): 18,
 ('캐럴', '크리스마스캐럴'): 3,
 ('캐럴', '겨울왕국'): 2,
 ('분위기', '겨울노래'): 6,
 ('분위기', '크리스마스송'): 1,
 ('분위기', '눈오는날'): 6,
 ('분위기', '크리스마스'): 50,
 ('분위기', '캐럴'): 6,
 ('분위기', '연말'): 36,
 ('분위기', '따듯한'): 3,
 ('분위기', '크리스마스캐럴'): 2,
 ('분위기', '겨울왕국'): 1,
 ('따듯한', '겨울노래'): 7,
 ('따듯한', '크리스마스송'): 1,
 ('따듯한', '눈오는날'): 5,
 ('따듯한', '크리스마스'): 14,
 ('따듯한', '분위기'): 3,
 ('따듯한', '캐럴'): 3,
 ('따듯한', '연말'): 12,
 

In [68]:
item_to_cnt = {}
all_cnt = 0
for i in tqdm_notebook(trans_id_idx.index.unique()): # 각 플스트 아이디에서
    items = trans_id_idx.loc[i] # 그 플스트에 수록된 태그들
    if type(items) is not str:
        all_cnt += 1
        for j in set(items): # 고유 태그들
            item_to_cnt[j] = item_to_cnt.get(j, 0) + 1
item_to_cnt # 전체 플레이리스트에서 총 태깅된 횟수

{'추억': 6488,
 '회상': 4578,
 '잔잔한': 9989,
 '까페': 2635,
 '겨울노래': 206,
 '크리스마스송': 5,
 '눈오는날': 107,
 '크리스마스': 936,
 '분위기': 2466,
 '캐럴': 75,
 '연말': 466,
 '따듯한': 112,
 '크리스마스캐럴': 7,
 '겨울왕국': 34,
 '2017': 101,
 '팝': 3278,
 '드라이브': 9839,
 '힐링': 9760,
 '기분전환': 15808,
 '운동': 2549,
 '일렉': 735,
 'Pop': 3021,
 '트로피컬하우스': 70,
 '트렌드': 131,
 '고백': 494,
 '취향저격': 1341,
 '사랑': 8654,
 '이별': 5349,
 '짝사랑': 265,
 '슬픔': 3520,
 '포크': 345,
 '일렉트로니카': 909,
 '인디': 3626,
 '댄스': 1735,
 '메탈': 326,
 '락': 1030,
 'Rock': 622,
 'Metal': 94,
 '록': 703,
 '이일우': 1,
 'M에센셜': 125,
 '스트레스해소': 210,
 '걸그룹댄스': 10,
 'kpop': 304,
 '카카오톡': 14,
 '카톡': 7,
 '프로필': 8,
 '프로필음악': 4,
 '소원': 9,
 '다짐': 9,
 '새해': 192,
 '소망': 7,
 '여행': 4535,
 '힘내': 130,
 '우울': 1122,
 '이거': 1,
 '듣고': 4,
 '감각적인': 867,
 '밤': 7598,
 '국내': 272,
 '느낌있는': 789,
 '새벽': 8284,
 '힙합': 6281,
 'RnB': 951,
 '그루브한': 46,
 '가을': 3822,
 '재즈': 2327,
 '감성': 11309,
 '질리지않는': 17,
 '나만알고싶은': 148,
 '봄': 3264,
 '설렘': 4904,
 '누군가생각날때': 1,
 '비오는날': 2829,
 '올림픽': 19,
 '폐막식': 4,
 '엑소': 96,

## 무조건 지지도가 높은 순으로 추천해주는 게 더 성능이 좋음
지지도(support) s(X→Y) 
= X와 Y를 모두 포함하는 거래 수 / 전체 거래 수 = n(X∪Y) / N

In [84]:
upgraded_dict = {}
for t in tqdm_notebook(item_tuple_to_cnt.keys()):
    if item_tuple_to_cnt[t] >= 1: # 1번 이상 다른 것과 함께 태깅된 태그만
        upgraded_dict[t] = (item_tuple_to_cnt[t] / all_cnt , item_tuple_to_cnt[t])
        # 둘이 같이 나온 횟수를 전체 등장 횟수로 나눔

In [83]:
all_cnt

87829

In [85]:
upgraded_dict

{('추억', '회상'): (0.04934588803242665, 4334),
 ('회상', '추억'): (0.04934588803242665, 4334),
 ('까페', '잔잔한'): (0.003700372314383632, 325),
 ('잔잔한', '까페'): (0.003700372314383632, 325),
 ('연말', '겨울노래'): (0.0003757301119220303, 33),
 ('연말', '크리스마스송'): (1.1385760967334253e-05, 1),
 ('연말', '눈오는날'): (0.00012524337064067677, 11),
 ('연말', '크리스마스'): (0.0021177515399241707, 186),
 ('연말', '분위기'): (0.00040988739482403306, 36),
 ('연말', '캐럴'): (0.00020494369741201653, 18),
 ('연말', '따듯한'): (0.000136629131608011, 12),
 ('연말', '크리스마스캐럴'): (1.1385760967334253e-05, 1),
 ('연말', '겨울왕국'): (0.00011385760967334252, 10),
 ('눈오는날', '겨울노래'): (0.00012524337064067677, 11),
 ('눈오는날', '크리스마스송'): (1.1385760967334253e-05, 1),
 ('눈오는날', '크리스마스'): (0.00040988739482403306, 36),
 ('눈오는날', '분위기'): (6.83145658040055e-05, 6),
 ('눈오는날', '캐럴'): (2.2771521934668505e-05, 2),
 ('눈오는날', '연말'): (0.00012524337064067677, 11),
 ('눈오는날', '따듯한'): (5.692880483667126e-05, 5),
 ('눈오는날', '크리스마스캐럴'): (1.1385760967334253e-05, 1),
 ('눈오는날', '겨울왕국'):

In [86]:
def item_reco1(tag):
    #name = items_.loc[song_id, 'song_name']
    cnt = 0
    print('현재태그 : {}'.format(tag))
    print('연관태그, 연관지수')
    for i in sorted(upgraded_dict.items(), key=lambda x : -x[1][0]):
        if i[0][0] == str(tag) and i[0][1] != str(tag): # 똑같은 태그 두 개는 어쨌든 연관지수가 1이니.
            print('{}, {:.2f}'.format(i[0][1],
                                      i[1][0]))
            cnt += 1
            if cnt == 100: # 100개만 추천해주는 것
                break
    if cnt == 0:
        print('5번 이상 동시 태깅된 태그가 없습니다.')
# 연관지수==향상도(lift)

In [87]:
item_reco1('잔잔한')

현재태그 : 잔잔한
연관태그, 연관지수
감성, 0.04
휴식, 0.03
새벽, 0.03
힐링, 0.02
밤, 0.02
기분전환, 0.02
카페, 0.02
발라드, 0.01
사랑, 0.01
겨울, 0.01
이별, 0.01
추억, 0.01
매장음악, 0.01
인디, 0.01
가을, 0.01
위로, 0.01
드라이브, 0.01
비오는날, 0.01
뉴에이지, 0.01
피아노, 0.01
봄, 0.01
회상, 0.01
연주곡, 0.00
슬픔, 0.00
취향저격, 0.00
분위기, 0.00
센치, 0.00
설렘, 0.00
커피, 0.00
여행, 0.00
팝, 0.00
까페, 0.00
편안한, 0.00
자장가, 0.00
새벽감성, 0.00
우울, 0.00
어쿠스틱, 0.00
잠들기전, 0.00
신나는, 0.00
Pop, 0.00
아침, 0.00
클래식, 0.00
주말, 0.00
조용한, 0.00
공부, 0.00
산책, 0.00
스트레스, 0.00
재즈, 0.00
따뜻한, 0.00
집중, 0.00
비, 0.00
독서, 0.00
여름, 0.00
오후, 0.00
팝송, 0.00
혼자, 0.00
퇴근길, 0.00
저녁, 0.00
그리움, 0.00
여유, 0.00
알앤비, 0.00
불면증, 0.00
매장, 0.00
숙면, 0.00
차분한, 0.00
출근길, 0.00
일상, 0.00
OST, 0.00
감성적인, 0.00
몽환, 0.00
힙합, 0.00
포근한, 0.00
명곡, 0.00
카페음악, 0.00
여름밤, 0.00
눈물, 0.00
커피숍, 0.00
눈, 0.00
외로움, 0.00
인디음악, 0.00
추위, 0.00
태교, 0.00
잠, 0.00
명상, 0.00
여유로운, 0.00
쓸쓸한, 0.00
북카페, 0.00
음색, 0.00
느낌있는, 0.00
가을밤, 0.00
밤새벽, 0.00
RnB, 0.00
감미로운, 0.00
집, 0.00
감성발라드, 0.00
겨울밤, 0.00
배경음악, 0.00
생각, 0.00
포크, 0.00
Jazz, 0.00


In [88]:
item_reco1('트로피컬하우스')

현재태그 : 트로피컬하우스
연관태그, 연관지수
EDM, 0.00
여름, 0.00
기분전환, 0.00
하우스, 0.00
일렉트로니카, 0.00
드라이브, 0.00
딥하우스, 0.00
신나는, 0.00
트로피컬, 0.00
시원한, 0.00
매장음악, 0.00
여행, 0.00
일렉, 0.00
청량, 0.00
댄스, 0.00
청량한, 0.00
일렉트로닉, 0.00
트로피칼하우스, 0.00
일렉트로팝, 0.00
시원, 0.00
팝, 0.00
electronica, 0.00
청량함, 0.00
청량감, 0.00
힐링, 0.00
운동, 0.00
Pop, 0.00
스트레스, 0.00
퓨쳐하우스, 0.00
프로그레시브하우스, 0.00
커머셜, 0.00
퓨처하우스, 0.00
트렌디, 0.00
카이고, 0.00
TropicalHouse, 0.00
편집샵, 0.00
산책, 0.00
카페, 0.00
감성, 0.00
새벽, 0.00
힙합, 0.00
아이돌, 0.00
취향저격, 0.00
더위, 0.00
드라이브할때신나는노래, 0.00
일렉트로닉팝, 0.00
휴가, 0.00
통통, 0.00
해변, 0.00
Fresh, 0.00
사이다, 0.00
DJ, 0.00
kpop, 0.00
세련된, 0.00
이디엠, 0.00
팝송, 0.00
트로피칼사운드, 0.00
2017, 0.00
트렌드, 0.00
세부장르, 0.00
house, 0.00
여행휴식, 0.00
2010, 0.00
라운지, 0.00
빌보드차트, 0.00
라운지음악, 0.00
감각적인, 0.00
신스팝, 0.00
해외, 0.00
매력적인, 0.00
일렉팝, 0.00
퓨처베이스, 0.00
commercial, 0.00
전자음악, 0.00
가을, 0.00
트랜스, 0.00
프로그레시브, 0.00
그루브, 0.00
일상, 0.00
중독성있는, 0.00
일렉2020, 0.00
Kygo, 0.00
켈빈해리스, 0.00
신나, 0.00
요즘대세, 0.00
프로그래시브하우스, 0.00
꿀렁꿀렁, 0.00
편안한, 0.00
EDM추천, 0.00
봄,

In [ ]:
# 트로피컬하우스 = 트로피칼하우스, TropicalHouse 등등 같이 묶이는 걸 보면 굳이 전처리 안해도 될듯